https://python.ivan-shamaev.ru/5-useful-functions-of-pandas-python-for-data-science/

In [ ]:
import pandas as pd
import numpy as np
import warnings # отключим предупреждения Anaconda
warnings.simplefilter('ignore')
%matplotlib inline 
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg' #графики в svg выглядят более четкими
from pylab import rcParams #увеличим дефолтный размер графиков
plt.style.use('ggplot')  # Красивые графики
plt.rcParams['figure.figsize'] = (15, 5)  # Размер картинок
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go
import plotly.graph_objects as go

init_notebook_mode(connected=True)
pd.options.display.float_format = '{:,.1f}'.format # Отображение чисел float с запятыми и 1 знаком после точки

f500 = pd.read_csv("f500.csv", index_col=0)

In [ ]:
f500[['sector', 'industry']].value_counts()
bottom_companies = f500.loc["National Grid":"AutoNation", ["rank", "sector", "country"]]
big_movers = f500.loc[["Aviva", "HP", "JD.com", "BHP Billiton"], ["rank", "previous_rank"]]
rank_change = f500['previous_rank'] - f500['rank']
rank_change.max()

In [ ]:
f500.describe() #Общее описание всех цифровых колонок
f500["rank"].describe() #Статистика колонки
f500["previous_rank"].value_counts().loc[0] #Ищем определенные параметры в столбце
f500[["revenues", "profits"]].median(axis=0) #AXIS 0 ищет по всем индексам, 1 - по всем колонкам
f500.max(numeric_only=True) #Ищет только по числовым колонкам
f500.describe(include=['O']) #Смотрит только на колонки с объектами

In [ ]:
new_f500 = f500['previous_rank'] == 0 #Находим строки с пробелами в данных
f500_selection = f500[['rank', 'revenues', 'revenue_change']]
f500.loc[new_f500, 'country'].value_counts()
f500["previous_rank"].value_counts(dropna=False).head() #Дропна не позволяет просто отборсить пустые ячейки 
f500.loc[f500["previous_rank"] == 0, "previous_rank"] = np.nan #Заменяем на на NaN. Что превращает инт в флоат
f500["industry"][f500["country"] == "USA"].value_counts() #Сортировка по значениям в колонках и индексах

In [ ]:
f500 = pd.read_csv("f500.csv")
f500.loc[f500["previous_rank"] == 0, "previous_rank"] = np.nan
first_column = f500.iloc[:,0] #ILOC это индексация построенная на цифровых значениях
slices = f500.iloc[:, [1,5]]

In [ ]:
#Метод выбирает индексы со значением ноль в ячейках
null_previous_rank = f500[f500["previous_rank"].isnull()][["company","rank", "previous_rank"]] #Здесь нули
first_null_prev_rank = null_previous_rank.iloc[0]
previously_ranked = f500[f500["previous_rank"].notnull()] #Здесь не нули
rank_change = previously_ranked["previous_rank"] - previously_ranked["rank"] #работа с ненулевыми значениями
f500["rank_change"] = rank_change #Добавление колонки
print(rank_change.shape)
print(rank_change.tail(3))

In [ ]:
# Выборка по заданным булевым параметрам
big_rev_neg_profit = (f500['revenues'] > 100000) & (f500['profits'] < 0)
filter_brazil_venezuela = (f500["country"] == "Brazil") | (f500["country"] == "Venezuela")
filter_tech_outside_usa = (f500["sector"] == "Technology") & ~(f500["country"] == "USA")

In [ ]:
#сортировка по доп. параметрам
f500[f500["country"] == "China"].sort_values("employees", ascending=False)
f500[f500["country"] == "Japan"].sort_values("employees", ascending=False).iloc[0]['company']

#Сортировка с использованием циклов
avg_rev_by_country = {}
countries = f500["country"].unique()
for c in countries:    
    selected_rows = f500[f500["country"] == c]
    mean = selected_rows["revenues"].mean()
    avg_rev_by_country[c] = mean
    
top_employer_by_country = {}
countries = f500["country"].unique()
for c in countries:
    selected_rows = f500[f500["country"] == c]
    sorted_rows = selected_rows.sort_values("employees", ascending=False)
    top_employer = sorted_rows.iloc[0]
    employer_name = top_employer["company"]
    top_employer_by_country[c] = employer_name
    
f500['roa'] = f500['profits'] / f500['assets']
top_roa_by_sector = {}
for sector in f500["sector"].unique(): #Выделяем группы
    is_sector = f500["sector"] == sector 
    sector_companies = f500.loc[is_sector] #Разбиваем компании по этим группам
    top_company = sector_companies.sort_values("roa",ascending=False).iloc[0] #Находим топ в списке и находим ее название
    top_roa_by_sector[sector] = top_company["company"] #ЗЗаносим ключ и значение в словарь

In [ ]:
laptops = pd.read_csv('laptops.csv', encoding='Latin-1')

In [ ]:
#Чистим заголовки от пробелов
def clean_col(col):
    col = col.strip() #Разделяем и убираем мусор
    col = col.lower()
    col = col.replace("operating_system","os")
    col = col.replace("(","")
    col = col.replace(")","")    
    col = col.replace(" ","_")    
    return col

new_columns = []
for col in laptops.columns:
    col = clean_col(col)    
    new_columns.append(col)

laptops.columns = new_columns #Создаем очищенный список, который подставляем на место прошлых имен колонок

In [ ]:
laptops.columns

In [ ]:
unique_ram = laptops["ram"].unique()
laptops["ram"] = laptops["ram"].str.replace('GB','')
unique_ram = laptops["ram"].unique()
laptops["ram"] = laptops["ram"].str.replace('GB','')
laptops["ram"] = laptops["ram"].astype(int)
dtypes = laptops.dtypes
laptops["ram"] = laptops["ram"].str.replace('GB','').astype(int)
laptops.rename({"ram": "ram_gb"}, axis=1, inplace=True)
ram_gb_desc = laptops["ram_gb"].describe()
laptops["gpu_manufacturer"] = (laptops["gpu"]
                                       .str.split()
                                       .str[0]
                              )
laptops["cpu_manufacturer"] = (laptops["cpu"]
                                       .str.split()
                                       .str[0]
                               )
cpu_manufacturer_counts = laptops["cpu_manufacturer"].value_counts()

mapping_dict = {
    'Android': 'Android',
    'Chrome OS': 'Chrome OS',
    'Linux': 'Linux',
    'Mac OS': 'macOS',
    'No OS': 'No OS',
    'Windows': 'Windows',
    'macOS': 'macOS'
}
laptops["os"] = laptops["os"].map(mapping_dict)

laptops_no_null_rows = laptops.dropna(axis=0)
laptops_no_null_cols = laptops.dropna(axis=1)

value_counts_before = laptops.loc[laptops["os_version"].isnull(), "os"].value_counts()
laptops.loc[laptops["os"] == "macOS", "os_version"] = "X"
laptops.loc[laptops["os"] == "No OS", "os_version"] = "Version Unknown"
value_counts_after = laptops.loc[laptops["os_version"].isnull(), "os"].value_counts()

laptops["weight"] = laptops["weight"].str.replace("kgs","").str.replace("kg","").astype(float)
laptops.rename({"weight": "weight_kg"}, axis=1, inplace=True)
laptops.to_csv('laptops_cleaned.csv',index=False)

https://github.com/dataquestio/solutions/blob/master/Mission294Solutions.ipynb

In [ ]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')
autos.info()
autos.head()

In [ ]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen']

In [ ]:
autos.describe(include='all')

In [ ]:
autos["num_photos"].value_counts()

In [ ]:
autos = autos.drop(["num_photos", "seller", "offer_type"], axis=1)

In [ ]:
autos["price"] = (autos["price"]
                          .str.replace("$","")
                          .str.replace(",","")
                          .astype(int)
                          )
autos["price"].head()

In [ ]:
autos["odometer"] = (autos["odometer"]
                             .str.replace("km","")
                             .str.replace(",","")
                             .astype(int)
                             )
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)
autos["odometer_km"].head()